# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Configure the API Key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('..\Output_Data\Weather_Data_Collection.csv')
weather_df = weather_df.drop(columns={'Unnamed: 0'})
weather_df.head()



,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,atuona,-9.80,-139.03,25.76,78,4,10.65,PF,1601619564
1,klaksvik,62.23,-6.59,13.00,76,89,8.70,FO,1601619844
2,kodiak,57.79,-152.41,10.00,87,90,11.30,US,1601619845
3,tasiilaq,65.61,-37.64,2.00,74,100,12.90,GL,1601619818
4,kabompo,-13.59,24.20,24.93,39,0,2.01,ZM,1601619845


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Obtaining the Coordinates to plot on the map
city_locations = weather_df[['Lattitude','Longitude']].astype(float)
city_locations.head()

,Lattitude,Longitude
0,-9.80,-139.03
1,62.23,-6.59
2,57.79,-152.41
3,65.61,-37.64
4,-13.59,24.20


In [4]:
# Humidity values
humidity_data = weather_df['Humidity']
humidity_data.head()

0    78
1    76
2    87
3    74
4    39
Name: Humidity, dtype: int64

In [5]:
# Creating a Humidity Heatmap Layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(city_locations, weights=humidity_data, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(humidity_layer)

fig



Figure(layout=FigureLayout(height=&#39;420px&#39;))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# The most ideal vacations involve destinations with minimal cloud cover as well as having temperatures within the range of 20-30 degrees celcius. As such, there datframe will be narrowed down further in order to find hotels within these regions that have the ideal weather. 

ideal_weather_df = weather_df.loc[
    (weather_df['Cloudiness']==0) & 
    (weather_df['Wind Speed'] <= 10) & 
    (weather_df['Max Temp'] >= 20) &
    (weather_df['Max Temp'] <= 30)].dropna()

print(f' The Number of Ideal Cities within the ideal weather criteria are {len(ideal_weather_df)}')
ideal_weather_df


 The Number of Ideal Cities within the ideal weather criteria are 24


,City,Lattitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,kabompo,-13.59,24.20,24.93,39,0,2.01,ZM,1601619845
31,sanandaj,35.31,46.99,22.57,27,0,1.24,IR,1601619852
55,redcliffe,-27.23,153.10,26.11,50,0,4.60,AU,1601619857
123,jalu,29.03,21.55,26.67,37,0,4.71,LY,1601619877
134,basco,20.45,121.97,27.82,70,0,7.52,PH,1601619885
145,ulladulla,-35.35,150.47,27.00,22,0,5.10,AU,1601619892
148,flinders,-34.58,150.86,24.44,54,0,1.34,AU,1601619894
206,abu samrah,24.60,72.70,26.66,52,0,2.28,IN,1601619678
211,gerash,27.67,54.14,30.00,30,0,0.43,IR,1601619940
235,wenling,28.37,121.36,21.67,64,0,2.95,CN,1601619956


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Creating a New Hotel DataFram
hotel_df = ideal_weather_df.copy().drop(columns={'Date', 'Wind Speed','Max Temp','Humidity','Cloudiness'})

hotel_df['Hotel Name'] = ""

hotel_df


,City,Lattitude,Longitude,Country,Hotel Name
4,kabompo,-13.59,24.20,ZM,
31,sanandaj,35.31,46.99,IR,
55,redcliffe,-27.23,153.10,AU,
123,jalu,29.03,21.55,LY,
134,basco,20.45,121.97,PH,
145,ulladulla,-35.35,150.47,AU,
148,flinders,-34.58,150.86,AU,
206,abu samrah,24.60,72.70,IN,
211,gerash,27.67,54.14,IR,
235,wenling,28.37,121.36,CN,


In [8]:
# Setting up the parameters to determine the closest hotels within a 5 km radium (5000 m)

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Setting up the search parameters
params = {
    'type':'hotel',
    'keyword': 'hotel',
    'radius': 5000,
    'key': g_key
}

# We will be itterating through the rows in order to hotels within the defined radius of the coordinates of each city
print(f'Data retrival in progress')
for index, row in hotel_df.iterrows():
    latitude = row['Lattitude']
    longitude = row['Longitude']
    city = row['City']

    # Now that the latitude and longitude has been defined, these need to be added to the dictionary

    params['location'] = f'{latitude},{longitude}'

    
    response = requests.get(base_url, params=params).json()

    search_results = response['results']

    try:
        print(f'Finding nearest hotel for {city}\n--------------------------')
        hotel_df.loc[index, 'Hotel Name'] = search_results[0]['name']
    
    except (KeyError, IndexError):
        print(f'There are no Hotels for {city}, moving on to the next city....\n')


print('End of Data Retreival\n*********************')
print(f'Removing cities where hotels could not be retreived')
hotel_df = hotel_df[hotel_df['Hotel Name']!=""]




Data retrival in progress
Finding nearest hotel for kabompo
--------------------------
Finding nearest hotel for sanandaj
--------------------------
Finding nearest hotel for redcliffe
--------------------------
Finding nearest hotel for jalu
--------------------------
Finding nearest hotel for basco
--------------------------
Finding nearest hotel for ulladulla
--------------------------
Finding nearest hotel for flinders
--------------------------
Finding nearest hotel for abu samrah
--------------------------
Finding nearest hotel for gerash
--------------------------
Finding nearest hotel for wenling
--------------------------
Finding nearest hotel for ajdabiya
--------------------------
Finding nearest hotel for najran
--------------------------
There are no Hotels for najran, moving on to the next city....

Finding nearest hotel for cockburn town
--------------------------
Finding nearest hotel for ginda
--------------------------
Finding nearest hotel for ridgecrest
------------

In [9]:
# Results of the Hotel Search
hotel_df

,City,Lattitude,Longitude,Country,Hotel Name
4,kabompo,-13.59,24.20,ZM,Chidikumbidi Lodge
31,sanandaj,35.31,46.99,IR,hedayat hotel
55,redcliffe,-27.23,153.10,AU,Oaks Redcliffe Mon Komo Suites
123,jalu,29.03,21.55,LY,Jalu Hotel
134,basco,20.45,121.97,PH,Villa Hontomin
145,ulladulla,-35.35,150.47,AU,Bannisters Pavilion Rooftop Bar & Grill
148,flinders,-34.58,150.86,AU,Lakeview Hotel Motel
206,abu samrah,24.60,72.70,IN,Hotel Hilltone
211,gerash,27.67,54.14,IR,Chelpasin
235,wenling,28.37,121.36,CN,Wenling International Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
hotel_marker = gmaps.marker_layer(locations, info_box_content=hotel_info)

# We will add the new layer to the previous humidity heatmap
fig.add_layer(hotel_marker)

# Display figure
fig

Figure(layout=FigureLayout(height=&#39;420px&#39;))